In [1]:
#it will say how many layers are needed and how many neurons are there
!pip install keras-tuner


     |████████████████████████████████| 98 kB 2.3 MB/s 


In [2]:
import tensorflow  as tf
from tensorflow import keras
import numpy as np 

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
#fashion mnist images are of grey scale which means every pixel values are from  0-255, now we scale them between 0 and 1
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [24]:
train_images[0].shape

(28, 28, 1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    #first convolutional layer
    #we can give max number of neurons in that hiddden layer 1 is 128 and min is 32, we do hyper parametr tuning using keras tuner
    #kernel size is notthing but filter size 
    #once we do convolution we activate it by relau activation
    #since being a first conv layer we have to give the input shape.
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    #out put layer with dimension 10
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [12]:
#randomsearch cv does the hyper parametr tuning and gives the best number number of filters, filetr size for every layer 
tuner_search=RandomSearch(build_model,objective='val_accuracy', max_trials=3, directory='output',project_name='fashion')

In [13]:
#for the random search object  we pass x_train and y_train to get the best paramtres
tuner_search.search(train_images,train_labels,epochs=2, validation_split=0.1)

Trial 3 Complete [00h 00m 42s]
val_accuracy: 0.8615000247955322

Best val_accuracy So Far: 0.8711666464805603
Total elapsed time: 00h 02m 06s
INFO:tensorflow:Oracle triggered exit


In [21]:
#saving the best model as below
model=tuner_search.get_best_models(num_models=1)[0]

In [28]:
#if you want to see tthe best parametsr 
tuner_search.get_best_hyperparameters()[0].values

{'conv_1_filter': 64,
 'conv_1_kernel': 5,
 'conv_2_filter': 32,
 'conv_2_kernel': 3,
 'dense_1_units': 32,
 'learning_rate': 0.01}

In [25]:
#with that best modeel we fit and train 
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.3368 - accuracy: 0.8759 - val_loss: 0.3688 - val_accuracy: 0.8637
Epoch 5/10
1688/1688 [==============================] - 14s 9ms/step - loss: 0.3153 - accuracy: 0.8818 - val_loss: 0.3531 - val_accuracy: 0.8688
Epoch 6/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2980 - accuracy: 0.8884 - val_loss: 0.3523 - val_accuracy: 0.8728
Epoch 7/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2972 - accuracy: 0.8894 - val_loss: 0.3813 - val_accuracy: 0.8640
Epoch 8/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2738 - accuracy: 0.8984 - val_loss: 0.3728 - val_accuracy: 0.8682
Epoch 9/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2589 - accuracy: 0.9033 - val_loss: 0.3868 - val_accuracy: 0.8718
Epoch 10/10
1688/1688 [==============================] - 14s 9ms/step - loss: 0.2516 - accuracy: 0.9063 - val_loss: 0.4894 - val_a